In [1]:
import time
from collections import defaultdict
from app.factor.factorModel import factorModel  
import pandas as pd
import numpy as np

import pickle

self = factorModel()

self.run()

Got data in 64.70107412338257 seconds
Processed in 4.5697479248046875 seconds
time used is 0.5209579467773438


{'group_0':     trade_date  dailyRet
 0     20210104  0.010287
 1     20210105  0.000112
 2     20210106 -0.004049
 3     20210107 -0.012736
 4     20210108  0.005757
 ..         ...       ...
 612   20230714  0.000343
 613   20230717 -0.002293
 614   20230718  0.003212
 615   20230719  0.003400
 616   20230720 -0.008119
 
 [617 rows x 2 columns],
 'group_1':     trade_date  dailyRet
 0     20210104  0.013891
 1     20210105  0.001959
 2     20210106 -0.007887
 3     20210107 -0.019622
 4     20210108  0.002802
 ..         ...       ...
 612   20230714  0.000169
 613   20230717 -0.001485
 614   20230718  0.002501
 615   20230719  0.001197
 616   20230720 -0.008100
 
 [617 rows x 2 columns],
 'group_2':     trade_date  dailyRet
 0     20210104  0.012395
 1     20210105  0.004557
 2     20210106 -0.007809
 3     20210107 -0.015589
 4     20210108  0.002670
 ..         ...       ...
 612   20230714  0.000781
 613   20230717 -0.001387
 614   20230718  0.000097
 615   20230719 -0.000074
 61

In [1]:
import time
from collections import defaultdict
from app.factor.factorModel import factorModel  
import pandas as pd
import numpy as np

import pickle

self = factorModel()

profit = self.run()
self.EachGroupPortRet(profit)


Got data in 60.62095808982849 seconds
Processed in 4.5708909034729 seconds
time used is 79.88080596923828


{'group_0':    trade_date  dailyRet
 0    20210104  0.010287
 1    20210105  0.000112
 2    20210106 -0.004049
 3    20210107 -0.012736
 4    20210108  0.005757
 ..        ...       ...
 9    20230714  0.000343
 10   20230717 -0.002293
 11   20230718  0.003212
 12   20230719  0.003400
 13   20230720 -0.008119
 
 [617 rows x 2 columns],
 'group_1':    trade_date  dailyRet
 0    20210104  0.013891
 1    20210105  0.001959
 2    20210106 -0.007887
 3    20210107 -0.019622
 4    20210108  0.002802
 ..        ...       ...
 9    20230714  0.000169
 10   20230717 -0.001485
 11   20230718  0.002501
 12   20230719  0.001197
 13   20230720 -0.008100
 
 [617 rows x 2 columns],
 'group_2':    trade_date  dailyRet
 0    20210104  0.012395
 1    20210105  0.004557
 2    20210106 -0.007809
 3    20210107 -0.015589
 4    20210108  0.002670
 ..        ...       ...
 9    20230714  0.000781
 10   20230717 -0.001387
 11   20230718  0.000097
 12   20230719 -0.000074
 13   20230720 -0.008771
 
 [617 rows 

In [19]:
#get runtime
import time
import pickle
from collections import defaultdict
from app.factor.factorModel import factorModel  
import pandas as pd
import numpy as np
self = factorModel()
import threading

with open('objs.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    Equity_Idx_Monthly_Equity_Returns, Monthly_Equity_Returns, Monthly_Factor_Score, Equity_Idx_Monthly_Factor_Score, Daily_Equity_Returns, benchmark_dailyret = pickle.load(f)

start_time = time.time()

try:    
    Daily_Equity_Returns = Daily_Equity_Returns.drop(columns=['trade_data'])
except:
    pass

Daily_Equity_Returns['trade_date'] = pd.to_datetime(Daily_Equity_Returns['trade_date'])

factor_names = list(Monthly_Factor_Score.keys())
month_names = list(Monthly_Equity_Returns.keys())
stock_names = list(Equity_Idx_Monthly_Factor_Score.keys())
minMonths = 2
maxMonths = 12
groupedProfit = defaultdict(dict)
totalIC = 0
combinedIC = {'month': [], 'IC': [], 'cumulative': []}

if self.factorWeightMode != 'smart':
    if self.factorWeightMode == 'equal':
        factorWeights = self.calcFactorWeights(self.factorWeightMode, factor_names)
    elif self.factorWeightMode == 'category':
        factorWeights = self.calcFactorWeights(self.factorWeightMode, factor_names, self.factorCategories)
elif self.factorWeightMode == 'smart':
    ICList = []

    for factor in factor_names:
        ICList.append(self.calcIC(month_names, Monthly_Factor_Score[factor], Monthly_Equity_Returns)[1])
    ICList = pd.DataFrame(ICList).T

    ICList.index = month_names
    ICList.columns = factor_names

#0.818s runtime

for month in range((minMonths if self.factorWeightMode == 'smart' else 0), len(month_names)):

    nameList, scoreList = [], []

    if self.factorWeightMode == 'smart':
        currList = ICList.loc[ICList.index[ICList.index <= month_names[month]]]
        if currList.shape[0] > maxMonths:
            currList = currList.iloc[-maxMonths:]
        factorWeights = self.calcFactorWeights(self.factorWeightMode, factor_names, HistoricalIC=currList, smartmode=self.factorWeightModeParams)
    else:
        if self.factorWeightMode == 'equal':
            factorWeights = self.calcFactorWeights(self.factorWeightMode, factor_names)
        elif self.factorWeightMode == 'category':
            factorWeights = self.calcFactorWeights(self.factorWeightMode, factor_names, self.factorCategories)

    #print(f'Weights - {factorWeights}')
    # print(factorWeights)
    for name in stock_names:
        name, score = self.calcEquityScore(name, factorWeights, Equity_Idx_Monthly_Factor_Score, month_names[month])
        if name:
            nameList.append(name)
            scoreList.append(score)
    
    equityGroups = self.rankEquity(nameList, scoreList)

    try:
        daily_returns_this_month = Daily_Equity_Returns[(Daily_Equity_Returns['trade_date'] >= month_names[month]) 
                            & (Daily_Equity_Returns['trade_date'] < month_names[month+1])]
    except:
        daily_returns_this_month = Daily_Equity_Returns[Daily_Equity_Returns['trade_date'] >= month_names[month]]
    
    for i, group in enumerate(equityGroups):
        # Filter df where 'ts_code' is in the current group
        df_group = daily_returns_this_month[daily_returns_this_month['ts_code'].isin(group)]
        groupedProfit[month_names[month]][f'group_{i}'] = df_group.reset_index(drop=True)    
    
    returnArr = []
    for stock in nameList:
        returnArr.append(Equity_Idx_Monthly_Equity_Returns[stock][month_names[month]])

    currIC = np.corrcoef(returnArr, scoreList)[0,1]
    
    totalIC += currIC
    combinedIC['month'].append(month_names[month])
    combinedIC['IC'].append(currIC)
    combinedIC['cumulative'].append(totalIC)

print("runtime: %s seconds" % (time.time() - start_time))

runtime: 1.2969942092895508 seconds


In [18]:
groupedProfit

defaultdict(dict,
            {'20210104': {'group_0':         ts_code  profit_daily trade_date
              0     000001.SZ     -0.038263 2021-01-04
              1     600345.SH      0.013590 2021-01-04
              2     600339.SH     -0.003247 2021-01-04
              3     600335.SH     -0.008048 2021-01-04
              4     600422.SH      0.002312 2021-01-04
              ...         ...           ...        ...
              4398  002761.SZ     -0.009296 2021-01-29
              4399  002621.SZ     -0.051188 2021-01-29
              4400  002589.SZ     -0.023109 2021-01-29
              4401  002663.SZ     -0.022989 2021-01-29
              4402  002654.SZ     -0.033233 2021-01-29
              
              [4403 rows x 3 columns],
              'group_1':         ts_code  profit_daily trade_date
              0     600361.SH      0.005025 2021-01-04
              1     600352.SH     -0.000734 2021-01-04
              2     600366.SH      0.034483 2021-01-04
              